In [24]:
import torch

In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q bitsandbytes datasets accelerate loralib
%pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 Ti (UUID: GPU-9b231919-e1e7-d9d8-f5cf-7c30056cb305)


In [6]:
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split



In [ ]:
import tarfile
import pickle
import io

In [8]:
# 1. Load the dataset
data = pd.read_csv('enron_data_fraud_labeled.csv')

C:\Users\golla\AppData\Local\Temp\ipykernel_47464\2480706587.py:2: DtypeWarning: Columns (7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('enron_data_fraud_labeled.csv')


In [9]:
data['Text'] = data['From'] + " " + data['Subject'] + " " + data['Body']

In [ ]:
df = data[['Text', 'Label']]

In [ ]:
# Open a tar file for writing
with tarfile.open('data.tar', 'w') as f:
    # Iterate over the DataFrame
    for i, row in df.iterrows():
        # Convert the 'Labels' and 'Text' fields to bytes using pickle
        label_bytes = pickle.dumps(row['Label'])
        text_bytes = pickle.dumps(row['Text'])
        
        # Create TarInfo objects for the data
        label_info = tarfile.TarInfo(name=f'{i}_label.pkl')
        label_info.size = len(label_bytes)
        text_info = tarfile.TarInfo(name=f'{i}_text.pkl')
        text_info.size = len(text_bytes)
        
        # Add the data to the tar file
        f.addfile(label_info, fileobj=io.BytesIO(label_bytes))
        f.addfile(text_info, fileobj=io.BytesIO(text_bytes))

In [14]:
# 2. Preprocess the data
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
data = data.dropna(subset=['Text', 'Label'])

train_texts, test_texts, train_labels, test_labels = train_test_split(data['Text'], data['Label'], test_size=0.2)


In [15]:
# Convert to list and tokenize
train_texts = train_texts.tolist()
test_texts = test_texts.tolist()


In [16]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [18]:
# 3. Load the pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\golla\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\golla\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model

In [41]:
torch.cuda.is_available()

False

In [38]:
from torch.utils.data import Dataset

class EncodingsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to tensor and add to encodings
#train_labels = train_labels.tolist()
#test_labels = test_labels.tolist()

train_dataset = EncodingsDataset(train_encodings, train_labels)
test_dataset = EncodingsDataset(test_encodings, test_labels)

# 4. Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)




# ... rest of your code ...

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()

  0%|          | 0/64410 [00:00<?, ?it/s]

C:\Users\golla\AppData\Local\Temp\ipykernel_47464\2062976411.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


KeyboardInterrupt: 

In [ ]:
# Convert labels to tensor and add to encodings
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())


train_encodings['labels'] = train_labels
test_encodings['labels'] = test_labels

# 4. Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_encodings,       # training dataset
    eval_dataset=test_encodings          # evaluation dataset
)

trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label,labels,label_ids.

In [26]:
# Convert labels to tensor and add to encodings
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())


train_encodings['labels'] = train_labels
test_encodings['labels'] = test_labels

# 4. Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_encodings,       # training dataset
    eval_dataset=test_encodings          # evaluation dataset
)

trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label,labels,label_ids.

In [ ]:
# 5. Evaluate the model
eval_results = trainer.evaluate()


In [ ]:
# 6. Save the model
model.save_pretrained('./my_fine_tuned_model')

In [17]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the dataset
data = pd.read_csv('enron_data_fraud_labeled.csv')

# Concatenate 'From', 'Subject' and 'Body' columns
data['Text'] = data['From'] + " " + data['Subject'] + " " + data['Body']

# 2. Preprocess the data
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_texts, test_texts, train_labels, test_labels = train_test_split(data['Text'], data['Label'], test_size=0.2)
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

# 3. Load the pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# 4. Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_encodings,         # training dataset
    eval_dataset=test_encodings             # evaluation dataset
)

trainer.train()

# 5. Evaluate the model
eval_results = trainer.evaluate()

# 6. Save the model
model.save_pretrained('./my_fine_tuned_model')

C:\Users\golla\AppData\Local\Temp\ipykernel_15912\2444473539.py:6: DtypeWarning: Columns (7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('enron_data_fraud_labeled.csv')
